In [1]:
import numpy as np
import sys
print(sys.version)

3.7.0 (v3.7.0:1bf9cc5093, Jun 26 2018, 23:26:24) 
[Clang 6.0 (clang-600.0.57)]


In [2]:
import logging.config
from sklearn.utils import shuffle

import numpy as np
from os import path

In [3]:
import quandl

In [4]:
import torch
from torch import nn
from torch import optim

In [5]:
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dropout, Dense, Activation
from tensorflow.keras import Sequential, callbacks
from tensorflow.keras.layers import Conv1D, MaxPooling1D
from tensorflow.keras.optimizers import RMSprop

In [6]:
import torchsummary

In [7]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from DataLoader import DataLoader
from MarketData import QuandlMarketDataSource, RedditMarketDataSource, BloombergMarketDataSource

from learning.BasicLearning import RbfClassifier
from utilities import Constants
from utilities.Utilities import Utilities
from MarketData import BloombergMarketDataSource

In [8]:
key_dir = "/Users/tianyudu/Documents/UToronto/Course/ECO499/quandl_api.txt"
with open(key_dir, "r") as f:
    key = f.readline()

In [9]:
from Experiment import *
from lstm_torch import BasicLSTM, Optimization

In [10]:
source = BloombergMarketDataSource()
stock="JPM"
price_source = "quandl"
sentiment_location = path.join(Constants.DATASETS_MARKET, "FinArticles/psenti/all.results.csv")

In [11]:
x_data, y_data = get_data(
    True,
    sentiment_location,
    price_source,
    stock
)

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.25)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.3)

create_x_tensor = lambda x: torch.from_numpy(np.expand_dims(x, axis=2)).float()
create_y_tensor = lambda y: torch.from_numpy(y).long()

(x_train, x_val, x_test) = map(create_x_tensor, (x_train, x_val, x_test))
(y_train, y_val, y_test) = map(create_y_tensor, (y_train, y_val, y_test))

2019-10-31 22:52:04,579 - utilities - INFO - Loading Stock [JPM]...


/Users/tianyudu/Documents/UToronto/Course/ECO499/ugthesis/replications/src/DataLoader.py:67: FutureWarning: pd.TimeGrouper is deprecated and will be removed; Please use pd.Grouper(freq=...)
  articles = articles.groupby(pd.TimeGrouper('D')).mean()
/Users/tianyudu/Documents/UToronto/Course/ECO499/ugthesis/replications/src/PortfolioBasic/stockstats.py:387: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  lambda x: np.fabs(x - x.mean()).mean())


In [22]:
model = BasicLSTM(input_size=1, hidden_size=400, output_size=2)
loss_fn = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=3e-3)
optimizer = optim.RMSprop(model.parameters(), lr=0.003, alpha=0.9)
# optimization = Optimization(model, loss_fn, optimizer)
optimization.train(
    x_train, y_train,
    x_val, y_val,
    n_epochs=300,
    batch_size=32
)

Epoch 1, Train CE: 0.678002, Train Acc: 0.572266. Val CE: 0.702335, Val Acc: 0.459821. Elapsed time: 1.52s.
Epoch 2, Train CE: 0.690026, Train Acc: 0.546875. Val CE: 0.686952, Val Acc: 0.558036. Elapsed time: 1.45s.
Epoch 3, Train CE: 0.686319, Train Acc: 0.556641. Val CE: 0.680782, Val Acc: 0.575893. Elapsed time: 1.40s.
Epoch 4, Train CE: 0.684968, Train Acc: 0.568359. Val CE: 0.687208, Val Acc: 0.526786. Elapsed time: 1.32s.
Epoch 5, Train CE: 0.686221, Train Acc: 0.544922. Val CE: 0.685865, Val Acc: 0.535714. Elapsed time: 1.37s.
Epoch 6, Train CE: 0.685700, Train Acc: 0.550781. Val CE: 0.686418, Val Acc: 0.535714. Elapsed time: 1.37s.
Epoch 7, Train CE: 0.684466, Train Acc: 0.550781. Val CE: 0.685381, Val Acc: 0.549107. Elapsed time: 1.34s.
Epoch 8, Train CE: 0.684032, Train Acc: 0.560547. Val CE: 0.687832, Val Acc: 0.531250. Elapsed time: 1.39s.
Epoch 9, Train CE: 0.685872, Train Acc: 0.554688. Val CE: 0.683374, Val Acc: 0.549107. Elapsed time: 1.40s.
Epoch 10, Train CE: 0.684217

KeyboardInterrupt: 

In [ ]:
optimization.plot_losses()

In [ ]:
optimization.plot_accuracies()